In [ ]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.2 MB/s eta 0:00:00


In [ ]:
!pip install -q wandb

In [ ]:
import os
from tokenizers import trainers, Tokenizer, models, pre_tokenizers
from datasets import load_dataset

In [ ]:
ds = load_dataset ("thainq107/iwslt2015-en-vi")

tokenizer_vi = Tokenizer(models.WordLevel(unk_token="<unk>"))
tokenizer_en = Tokenizer(models.WordLevel(unk_token="<unk>"))

tokenizer_en.pre_tokenizer = pre_tokenizers.Whitespace()
tokenizer_vi.pre_tokenizer = pre_tokenizers.Whitespace()

trainer = trainers.WordLevelTrainer(
    vocab_size=15000,
    min_frequency=2,
    special_tokens=['<pad>', '<unk>', '<bos>', '<eos>']
)

tokenizer_en.train_from_iterator(ds['train']['en'], trainer)
tokenizer_vi.train_from_iterator(ds['train']['vi'], trainer)

tokenizer_en.save('tokenizer_en.json')
tokenizer_vi.save('tokenizer_vi.json')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/522 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/181k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/133317 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1268 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1268 [00:00<?, ? examples/s]

In [ ]:
from transformers import PreTrainedTokenizerFast

tokenizer_en = PreTrainedTokenizerFast(
    tokenizer_file='tokenizer_en.json',
    unk_token='<unk>',
    pad_token='<pad>',
    bos_token='<bos>',
    eos_token='<eos>'
)

tokenizer_vi = PreTrainedTokenizerFast(
    tokenizer_file='tokenizer_vi.json',
    unk_token='<unk>',
    pad_token='<pad>',
    bos_token='<bos>',
    eos_token='<eos>'
)

MAX_LENGTH = 75

def preprocess_function(examples):
  src_texts = examples['en']
  tgt_texts = ['<bos>' + sent + '<eos>' for sent in examples['vi']]

  src_encoding = tokenizer_en(
      src_texts, padding='max_length', truncation=True, max_length=MAX_LENGTH
  )

  tgt_encoding = tokenizer_vi(
      tgt_texts, padding='max_length', truncation=True, max_length=MAX_LENGTH
  )

  return {
      'input_ids': src_encoding['input_ids'],
      'label_ids': tgt_encoding['input_ids']
  }

preprocessed_ds = ds.map(preprocess_function, batched=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Map:   0%|          | 0/133317 [00:00<?, ? examples/s]

Map:   0%|          | 0/1268 [00:00<?, ? examples/s]

Map:   0%|          | 0/1268 [00:00<?, ? examples/s]

In [ ]:
import torch
import torch.nn as nn
from transformers import PreTrainedModel, PretrainedConfig

class Seq2SeqRNNConfig(PretrainedConfig):
  def __init__(self, vocab_size_src=10000, vocab_size_tgt=10000,
               embedding_dim=128, hidden_dim=128, dropout=0.1, **kwargs):
    super().__init__(**kwargs)
    self.vocab_size_src = vocab_size_src
    self.vocab_size_tgt = vocab_size_tgt
    self.embedding_dim = embedding_dim
    self.hidden_dim = hidden_dim
    self.dropout = dropout


class EncoderRNN(nn.Module):
  def __init__(self, input_size, embedding_dim, hidden_size, dropout_p=0.1):
    super(EncoderRNN, self).__init__()
    self.input_size = input_size
    self.embedding = nn.Embedding(input_size, embedding_dim)
    self.gru = nn.GRU(embedding_dim, hidden_size, batch_first=True)
    self.dropout = nn.Dropout(dropout_p)

  def forward(self, input):
    embedded = self.dropout(self.embedding(input))
    output, hidden = self.gru(embedded)

    return output, hidden

class DecoderRNN(nn.Module):
  def __init__(self, hidden_size, embedding_dim, output_size):
    super(DecoderRNN, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(output_size, embedding_dim)
    self.gru = nn.GRU(embedding_dim, hidden_size, batch_first=True)
    self.out = nn.Linear(hidden_size, output_size)

  def forward(self, input, hidden):
    output = self.embedding(input)
    output, hidden = self.gru(output, hidden)
    output = self.out(output)

    return output, hidden

class Seq2SeqRNNModel(PreTrainedModel):
  def __init__(self, config, tokenizer_en):
    super().__init__(config)
    self.encoder = EncoderRNN(
        config.vocab_size_src, config.embedding_dim,
        config.hidden_dim, config.dropout
    )
    self.decoder = DecoderRNN(
        config.hidden_dim, config.embedding_dim, config.vocab_size_tgt
    )
    self.BOX_IDX = tokenizer_en.bos_token_id
    self.loss_fn = nn.CrossEntropyLoss(ignore_index=0)

  def forward(self, input_ids, labels):
    batch_size, seq_len = labels.shape
    decoder_input = torch.full((batch_size, 1), self.BOX_IDX, dtype=torch.long).to(input_ids.device)
    encoder_output, decoder_hidden = self.encoder(input_ids)
    decoder_outputs = []

    for i in range(seq_len):
      decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
      decoder_outputs.append(decoder_output)
      decoder_input = labels[:, i].unsqueeze(1)

    logits = torch.cat(decoder_outputs, dim=1)
    loss = self.loss_fn(logits.view(-1, logits.shape[-1]), labels.view(-1))

    return {'loss': loss, 'logits': logits}

config = Seq2SeqRNNConfig(
    vocab_size_src=len(tokenizer_en), vocab_size_tgt=len(tokenizer_vi)
)
model = Seq2SeqRNNModel(config, tokenizer_en)

In [ ]:
# input_ids = torch.tensor(preprocessed_ds['train'][0]['input_ids'])
# labels = torch.tensor(preprocessed_ds['train'][0]['labels'])

# pred = model(input_ids, labels)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./en-vi-machine_translator',
    logging_dir='logs',
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='epoch',
    per_device_train_batch_size=512,
    per_device_eval_batch_size=512,
    num_train_epochs=25,
    learning_rate=2e-5,
    save_total_limit=1,
    report_to='wandb',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_ds['train'],
    eval_dataset=preprocessed_ds['validation']
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tranhuysuxt (huytranduck) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,9.395700,9.231008
2,8.714300,7.781229
3,7.157200,6.764397
4,6.553100,6.407156
5,6.305100,6.237929
6,6.179800,6.147441
7,6.109000,6.094296
8,6.064500,6.058999
9,6.032900,6.032973
10,6.008300,6.012153


TrainOutput(global_step=6525, training_loss=6.277830291397271, metrics={'train_runtime': 2431.0067, 'train_samples_per_second': 1371.006, 'train_steps_per_second': 2.684, 'total_flos': 2944709228925000.0, 'train_loss': 6.277830291397271, 'epoch': 25.0})

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol, device="cpu"):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)

    for i in range(max_len - 1):
        memory = memory.to(device)
        tgt_mask = (generate_square_subsequent_mask(ys.size(1), device)
                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        prob = model.generator(out[:, -1, :])  # LM Head
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word[-1].item()  # index

        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)

        if next_word == 3:  # EOS: 3
            break

    return ys


def translate(model, src_sentence, device):
    model.eval()
    input_ids = tokenizer_en([src_sentence], return_tensors='pt')['input_ids'].to(device)
    num_tokens = input_ids.shape[1]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool).to(device)

    tgt_tokens = greedy_decode(
        model, input_ids, src_mask, max_len=num_tokens + 5, start_symbol=2, device=device
    )

    return tokenizer_vi.decode(tgt_tokens.detach().cpu()[0])


translate(model, "I go to school", model.device)  # => "Tôi đến trường"


# Evaluate on test set
from tqdm import tqdm
import sacrebleu

pred_sentences, tgt_sentences = [], []
for sample in tqdm(ds['test']):
    src_sentence = sample['en']
    tgt_sentence = sample['vi']

    pred_sentence = translate(model, src_sentence)
    pred_sentences.append(pred_sentence)
    tgt_sentences.append(tgt_sentence)

bleu_score = sacrebleu.corpus_bleu(pred_sentences, [tgt_sentences], force=True)

AttributeError: 'Seq2SeqRNNModel' object has no attribute 'encode'